In [1]:
import os

import re
import random
import xml.etree.ElementTree as et
from PIL import Image

import cv2
import numpy as np
import tensorflow as tf

from tensorflow import keras

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

print(tf.__version__)
print(keras.__version__)

2.5.2
2.5.0


In [4]:
!pip install gdown

In [2]:
import gdown 
url = 'https://drive.google.com/uc?id=1gXqmWrxJqdp_luNKZmv81vY5cjFLuTdT'
fname = 'oxford_pet.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gXqmWrxJqdp_luNKZmv81vY5cjFLuTdT
To: F:\SSAC\SSAC_2021_playdata\class_study\08.deeplearning\05\ch4\oxford_pet.zip
100%|███████████████████████████████████████████████████████████████████████████████| 811M/811M [00:28<00:00, 28.5MB/s]


'oxford_pet.zip'

In [37]:
from zipfile import ZipFile
ZipFile('oxford_pet.zip').extractall('oxford_pet')

In [5]:
#경로설정
base_dir = './'
image_dir = os.path.join(base_dir, 'images')
bbox_dir = os.path.join(base_dir, 'annotations', 'xmls')

In [8]:
print(image_dir)
print(bbox_dir)

./images
./annotations\xmls


In [12]:
os.path.splitext('abc.txt')[-1]

'.txt'

In [15]:
!dir

 F 드라이브의 볼륨: SAMSUNG
 볼륨 일련 번호: C60D-15BA

 F:\SSAC\SSAC_2021_playdata\class_study\08.deeplearning\05\ch4 디렉터리

2021-12-17  오후 04:46    <DIR>          .
2021-12-17  오후 04:46    <DIR>          ..
2021-12-17  오후 04:42    <DIR>          .ipynb_checkpoints
2021-12-17  오후 04:46            12,251 4강 개인.ipynb
2021-12-17  오후 04:44    <DIR>          oxford_pet
2021-12-17  오후 04:42       810,565,619 oxford_pet.zip
               2개 파일         810,577,870 바이트
               4개 디렉터리  1,836,353,974,272 바이트 남음


In [20]:
image_dir=os.path.join(base_dir, 'oxford_pet/images')

In [21]:
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1]=='.jpg']
print(len(image_files))

image_files[:3]

7390


['Abyssinian_1.jpg', 'Abyssinian_10.jpg', 'Abyssinian_100.jpg']

In [26]:
bbox_dir=os.path.join(base_dir, 'oxford_pet/annotations', 'xmls')
bbox_files = [fname for fname in os.listdir(bbox_dir) if os.path.splitext(fname)[-1] == '.xml']
print(len(bbox_files))
bbox_files[:3]

3686


['Abyssinian_1.xml', 'Abyssinian_10.xml', 'Abyssinian_100.xml']

In [33]:
image_dir, image_file

('./oxford_pet/images', 'Abyssinian_34.jpg')

In [38]:
remove_image_cnt = 0
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    bbox_file = os.path.splitext(image_file)[0]+'.xml'
    bbox_path = os.path.join(bbox_dir, bbox_file)
    
    image = Image.open(image_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        image = np.asarray(image)
        print(image_file, image_mode, image.shape)
        
        os.remove(image_path)
        remove_image_cnt +=1
        
        if os.path.isfile(bbox_path):
            print(bbox_path)
            os.remove(bbox_path)

Abyssinian_34.jpg P (202, 250)
Abyssinian_5.jpg RGBA (150, 200, 4)
Egyptian_Mau_129.jpg L (325, 299)
./oxford_pet/annotations\xmls\Egyptian_Mau_129.xml
Egyptian_Mau_139.jpg P (250, 350)
Egyptian_Mau_14.jpg RGBA (800, 582, 4)
Egyptian_Mau_145.jpg P (188, 216)
Egyptian_Mau_167.jpg P (275, 183)
Egyptian_Mau_177.jpg P (175, 246)
Egyptian_Mau_186.jpg RGBA (275, 183, 4)
Egyptian_Mau_191.jpg P (214, 300)
staffordshire_bull_terrier_2.jpg L (282, 500)
staffordshire_bull_terrier_22.jpg L (500, 364)


In [41]:
remove_image_cnt

12

In [42]:
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']
bbox_files = [fname for fname in os.listdir(bbox_dir) if os.path.splitext(fname)[-1] == '.xml']

In [47]:
print(image_files[:3])
bbox_files[:3]

['Abyssinian_1.jpg', 'Abyssinian_10.jpg', 'Abyssinian_100.jpg']


['Abyssinian_1.xml', 'Abyssinian_10.xml', 'Abyssinian_100.xml']

In [48]:
len(image_files), len(bbox_files)

(7390, 3686)

In [52]:
class_list = set()
for image_file in image_files:
    file_name = os.path.splitext(image_file)[0]
    #print('file_name', file_name)
    class_name = re.sub('_\d+', '', file_name)
    #print('class_name', class_name)
    class_list.add(class_name)
class_list = list(class_list)
class_list.sort()
class_list[:3]

['Abyssinian', 'Bengal', 'Birman']

In [55]:
class2idx = {cls:idx for idx,cls in enumerate(class_list)}
class2idx

{'Abyssinian': 0,
 'Bengal': 1,
 'Birman': 2,
 'Bombay': 3,
 'British_Shorthair': 4,
 'Egyptian_Mau': 5,
 'Maine_Coon': 6,
 'Persian': 7,
 'Ragdoll': 8,
 'Russian_Blue': 9,
 'Siamese': 10,
 'Sphynx': 11,
 'american_bulldog': 12,
 'american_pit_bull_terrier': 13,
 'basset_hound': 14,
 'beagle': 15,
 'boxer': 16,
 'chihuahua': 17,
 'english_cocker_spaniel': 18,
 'english_setter': 19,
 'german_shorthaired': 20,
 'great_pyrenees': 21,
 'havanese': 22,
 'japanese_chin': 23,
 'keeshond': 24,
 'leonberger': 25,
 'miniature_pinscher': 26,
 'newfoundland': 27,
 'pomeranian': 28,
 'pug': 29,
 'saint_bernard': 30,
 'samoyed': 31,
 'scottish_terrier': 32,
 'shiba_inu': 33,
 'staffordshire_bull_terrier': 34,
 'wheaten_terrier': 35,
 'yorkshire_terrier': 36}

In [56]:
Image_size = 224
N_bbox = len(bbox_files)
N_train = 3000
N_val = N_bbox-N_train

In [57]:
tfr_dir = os.path.join(base_dir, 'tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'oxford_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'oxford_val.tfr')

print(tfr_train_dir)
print(tfr_val_dir)

./tfrecord\oxford_train.tfr
./tfrecord\oxford_val.tfr


In [60]:
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

In [63]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value=[value]))

In [ ]:
shuffle_list = list(range(N_bbox))
random_shuffle(shuffle_list)

train_idx_list = shuffle_list[:N_train]
val_idx_list = shuffle_list[N_train:]

In [64]:
 type(tf.constant(0))

tensorflow.python.framework.ops.EagerTensor

In [ ]:
#train tfrecord 생성
for idx in train_idx_list:
    bbox_file = bbox_files[idx]
    bbox_path = os.path.join(bbox_dir, bbox_file)
    
    tree=et.parse(bbox_path)
    
    width = float(tree.find('./size/width').text)
    height = float(tree.find('./size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    xc = (xmin + ymax) / 2.
    yc = (ymin + ymax) / 2.
    
    